# Voice Changer (Colab) — Notebook pronto para GitHub

> **Descrição:** Notebook pronto para rodar no Google Colab (CPU-friendly). Ele fornece uma interface Gradio onde você pode enviar um áudio curto de referência (voz-alvo) e texto para sintetizar na voz clonada. Não é tempo real absoluto, mas funciona em *quase tempo real* em CPU com modelos leves.

---

## Instruções rápidas
1. Abra este notebook no Colab: `File -> Save a copy in GitHub` (ou `Ficheiro -> Guardar uma cópia no GitHub`).
2. Salve com o nome `voice_changer.ipynb` no seu repositório `voice-changer-colab`.
3. Execute as células na ordem. A primeira célula instala dependências (pode demorar).
4. Use a interface Gradio para enviar um arquivo de áudio curto (5–20s) como referência e um texto para sintetizar.

---

## Célula 1 — Instalar dependências
```bash
!pip install -q TTS==0.13.3 gradio soundfile librosa numpy scipy
```

## Célula 2 — Imports e preparação
```python
from TTS.api import TTS
import gradio as gr
import soundfile as sf
import numpy as np
import os

# Forçar uso de CPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""
```

## Célula 3 — Carregar modelo leve (padrão YourTTS/multilingual se disponível)
```python
model_name = "tts_models/multilingual/multi-dataset/your_tts"
tts = TTS(model_name, progress_bar=False, gpu=False)
print("Modelo carregado:", model_name)
```

## Célula 4 — Função de clonagem (usa arquivo de referência)
```python
def synthesize_clone(reference_wav_path, text, output_path="generated.wav", language="pt"):
    tts.tts_to_file(text=text, speaker_wav=reference_wav_path, language=language, file_path=output_path)
    return output_path
```

## Célula 5 — Interface Gradio
```python
def run_gradio():
    def process(speaker_audio, text, language):
        if speaker_audio is None or text.strip() == "":
            return None, "Envie um áudio de referência e texto."
        if isinstance(speaker_audio, tuple) or isinstance(speaker_audio, list):
            if isinstance(speaker_audio[0], np.ndarray):
                arr, sr = speaker_audio
                tmp_path = "_tmp_ref.wav"
                sf.write(tmp_path, arr, sr)
                reference_path = tmp_path
            else:
                reference_path = speaker_audio[0]
        elif isinstance(speaker_audio, str):
            reference_path = speaker_audio
        else:
            reference_path = "reference.wav"
            sf.write(reference_path, speaker_audio[0], speaker_audio[1])
        out = synthesize_clone(reference_path, text, output_path="output_clone.wav", language=language)
        return out, "Pronto: sintetizado em output_clone.wav"

    with gr.Blocks() as demo:
        gr.Markdown("# Voice Cloner — CPU (Colab)\nEnvie um áudio curto de referência (5–20s) e o texto a sintetizar.")
        with gr.Row():
            speaker = gr.Audio(type="numpy", label="Áudio de referência (wav)", source="upload")
            txt = gr.Textbox(label="Texto para sintetizar", lines=3, placeholder="Escreva o texto aqui...")
        lang = gr.Dropdown(choices=["pt", "en"], value="pt", label="Língua")
        btn = gr.Button("Gerar voz clonada")
        out_audio = gr.Audio(label="Áudio gerado")
        status = gr.Textbox(label="Status", interactive=False)

        btn.click(fn=process, inputs=[speaker, txt, lang], outputs=[out_audio, status])

    demo.launch(share=True)
```

## Observações e limitações
- **Sem GPU:** o tempo de geração aumenta.
- **Qualidade:** depende do modelo e da referência.
- **Ética:** clone apenas vozes com permissão.

